In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Anxiety_disorder"
cohort = "GSE94119"

# Input paths
in_trait_dir = "../../input/GEO/Anxiety_disorder"
in_cohort_dir = "../../input/GEO/Anxiety_disorder/GSE94119"

# Output paths
out_data_file = "../../output/preprocess/Anxiety_disorder/GSE94119.csv"
out_gene_data_file = "../../output/preprocess/Anxiety_disorder/gene_data/GSE94119.csv"
out_clinical_data_file = "../../output/preprocess/Anxiety_disorder/clinical_data/GSE94119.csv"
json_path = "../../output/preprocess/Anxiety_disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression and response to psychological therapy"
!Series_summary	"This study represents the first investigation of genome-wide expression profiles with respect to psychological treatment outcome. Participants (n=102) with panic disorder or specific phobia received exposure-based CBT. Treatment outcome was defined as percentage reduction from baseline in clinician-rated severity of their primary anxiety diagnosis at post-treatment and six month follow-up. Gene expression was determined from whole blood samples at 3 time-points using the Illumina HT-12v4 BeadChip microarray. No changes in gene expression were significantly associated with treatment outcomes when correcting for multiple testing (q<0.05), although a small number of genes showed a suggestive association with treatment outcome (q<0.5, n=20). Study reports suggestive evidence for the role of a small number of genes in treatment outcome. Although preliminary, the findings contribute

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this is a microarray study using Illumina HT-12v4 BeadChip
# for gene expression profiling, so gene expression data should be available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (anxiety disorder):
# The data doesn't explicitly state anxiety disorder status in the characteristics dictionary,
# but from the background information, we know all participants have either panic disorder or 
# specific phobia, which are types of anxiety disorders. 
# But there's no row key that distinguishes between different anxiety disorders or severity.
trait_row = None

# For age:
# There's no age information in the sample characteristics dictionary
age_row = None

# For gender:
# Gender is available at index 0
gender_row = 0

# 2.2 Data Type Conversion

# Since trait data is not available in a usable form for our analysis
def convert_trait(value):
    return None

# Since age data is not available
def convert_age(value):
    return None

# Convert gender to binary (0 for female, 1 for male)
def convert_gender(value):
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':')[1].strip()
    
    if value.upper() == 'FEMALE':
        return 0
    elif value.upper() == 'MALE':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, this dataset doesn't have the necessary trait data for our analysis,
# so we skip the clinical feature extraction step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651228', 'ILMN_1651254',
       'ILMN_1651262', 'ILMN_1651315', 'ILMN_1651347', 'ILMN_1651378',
       'ILMN_1651405', 'ILMN_1651680', 'ILMN_1651692', 'ILMN_1651705',
       'ILMN_1651719', 'ILMN_1651735', 'ILMN_1651788', 'ILMN_1651799',
       'ILMN_1651826', 'ILMN_1651832', 'ILMN_1651850', 'ILMN_1651886'],
      dtype='object', name='ID')

Gene data dimensions: 4381 genes × 315 samples


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers

# The identifiers starting with "ILMN_" are Illumina probe IDs, not direct human gene symbols
# These are probe identifiers used in Illumina microarray platforms and need to be mapped to human gene symbols
# for proper biological interpretation and cross-platform compatibility

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# Identify the columns in the gene annotation dataframe
# For the gene identifiers, we need 'ID' column which contains the ILMN_* probe IDs
# For gene symbols, we need the 'Symbol' column

# 1. Create a gene mapping dataframe with the probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# 2. Print a preview of the mapping dataframe to verify
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert the probe-level gene expression data to gene-level data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# 4. Print a preview of the gene expression data after mapping
print("\nGene data after mapping preview:")
print(f"Shape: {gene_data.shape}")
print(f"First 5 gene symbols: {list(gene_data.index[:5])}")


Gene mapping preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}

Gene data after mapping preview:
Shape: (3270, 315)
First 5 gene symbols: ['AARS', 'AATF', 'ABCF1', 'ABHD5', 'ABI1']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Use the NCBI gene synonym information to normalize gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data after normalization: shape {normalized_gene_data.shape}")
    print(f"First 5 normalized gene symbols: {normalized_gene_data.index[:5].tolist()}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized gene data for subsequent steps
    gene_data_final = normalized_gene_data
except Exception as e:
    print(f"Error during normalization: {e}")
    print("Using original gene data instead.")
    gene_data_final = gene_data
    
    # Save the original gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_final.to_csv(out_gene_data_file)
    print(f"Original gene data saved to {out_gene_data_file}")

# 2. Create clinical data with gender information (since trait data is unavailable)
if gender_row is not None:
    # Create a DataFrame with just gender information
    gender_data = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender)
    clinical_df = gender_data
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data_final)
    print(f"Linked data shape: {linked_data.shape}")
else:
    print("No clinical features available to link with gene expression data.")
    # Create a minimal DataFrame with gene expression data
    linked_data = gene_data_final.T  # Transpose to have samples as rows

# 3. Since trait data is unavailable, we can't perform trait-specific operations
# but we can still handle missing values in the gene expression data
is_trait_available = False
print("Trait data is unavailable in this dataset.")

# 4. Since trait data is unavailable, the dataset is not usable for trait association studies
is_biased = True  # Not applicable since trait is unavailable

# 5. Validate and save cohort info
note = "This dataset contains human anxiety disorder gene expression data, but lacks specific anxiety disorder trait measurements (e.g., severity scores) for association studies."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Don't save linked data as it's not usable for trait association studies
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for anxiety disorder association studies due to missing trait measurements.")

Gene data after normalization: shape (3207, 315)
First 5 normalized gene symbols: ['AARS1', 'AATF', 'ABCF1', 'ABHD5', 'ABI1']


Normalized gene data saved to ../../output/preprocess/Anxiety_disorder/gene_data/GSE94119.csv
Clinical data saved to ../../output/preprocess/Anxiety_disorder/clinical_data/GSE94119.csv
Linked data shape: (315, 3208)
Trait data is unavailable in this dataset.
Dataset deemed not usable for anxiety disorder association studies due to missing trait measurements.
